### Этап 1. Подготовка

In [ ]:
# импортируем библиотеки
import pandas as pd
import numpy as np
import re
import spacy
import nltk

from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score
from sklearn.model_selection import (
    GridSearchCV,
    train_test_split
)
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.dummy import DummyClassifier
from sklearn.pipeline import Pipeline

# зафиксируем значение random state
RS = 12345

# зафиксируем размер выборки
sample_size = 30000

In [ ]:
# загрузим данные из локальной папки
data = pd.read_csv('toxic_comments.csv')

# напишем функцию для проверки датасета
def check_data(df):
    display(
        df.info(),
        df.describe(),
        df.head()
    )
    print('Дубликатов строк:', df.duplicated().sum())

### Этап 2. Обзор данных

In [ ]:
# проверим датасет
check_data(data)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


None

,Unnamed: 0,toxic
count,159292.000000,159292.000000
mean,79725.697242,0.101612
std,46028.837471,0.302139
min,0.000000,0.000000
25%,39872.750000,0.000000
50%,79721.500000,0.000000
75%,119573.250000,0.000000
max,159450.000000,1.000000


,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


Дубликатов строк: 0


В данных:
- нет пропусков
- корректные типы
- нет дубликатов
- есть символы переноса строки \n, которые могут помешать обработке
- дисбаланс классов: токсичных комментариев около 10%
- есть неинформативный столбец Unnamed: 0

### Этап 3. Обработка данных

In [ ]:
# удалим неинформативный столбец
data = data.drop('Unnamed: 0', axis=1)

# для дальнейшей работы возьмем выборку из датасета
df = data.sample(sample_size, random_state=RS)

# проверим результат
df.shape

(30000, 2)

In [ ]:
# рассмотрим ближе тексты
df['text'].head(10).values

array(['Expert Categorizers  \n\nWhy is there no mention of the fact that Nazis were particularly great categorizers? They excelled in identifying various things and writing about them and putting them in their proper places.',
       '"\n\n Noise \n\nfart*  talk. "',
       'An indefinite block is appropriate, even for a minor infraction, if you show know signs of discontinuing.  —   (talk)',
       "I don't understand why we have a screenshot of AP's GUI but not UB. Can someone remedy this?",
       "Hello! Some of the people, places or things you have written about in the article Nikolas Tryfonos may not be sufficiently well-known to merit articles of their own. The Wikipedia community welcomes newcomers, and encourages them to become Wikipedians. On Wikipedia, each user is entitled to a user page in which they can describe themselves, and this article's content may be incorporated into that page. However, to merit inclusion in the encyclopedia proper, a subject must be notable. We 

In [ ]:
# напишем функцию для приведения слов к нижнему регистру и очистки от лишних символов
def clean(text):
    return " ".join(re.sub(r'[^a-zA-Z]\n', ' ', text.lower()).split())

# загрузим словарь spacy
nlp = spacy.load('en_core_web_sm')

# напишем функцию лемматизации
def lemmatize(text):
    global nlp
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc])

In [ ]:
%%time

# очистим и лемматизируем данные
df['clean_text'] = df['text'].apply(clean)
df['lemmatized_text'] = df['clean_text'].apply(lemmatize)

# проверим результат в сравнении
display(
    df['text'].head().values,
    df['clean_text'].head().values,
    df['lemmatized_text'].head().values
)

array(['Expert Categorizers  \n\nWhy is there no mention of the fact that Nazis were particularly great categorizers? They excelled in identifying various things and writing about them and putting them in their proper places.',
       '"\n\n Noise \n\nfart*  talk. "',
       'An indefinite block is appropriate, even for a minor infraction, if you show know signs of discontinuing.  —   (talk)',
       "I don't understand why we have a screenshot of AP's GUI but not UB. Can someone remedy this?",
       "Hello! Some of the people, places or things you have written about in the article Nikolas Tryfonos may not be sufficiently well-known to merit articles of their own. The Wikipedia community welcomes newcomers, and encourages them to become Wikipedians. On Wikipedia, each user is entitled to a user page in which they can describe themselves, and this article's content may be incorporated into that page. However, to merit inclusion in the encyclopedia proper, a subject must be notable. We 

array(['expert categorizers why is there no mention of the fact that nazis were particularly great categorizers? they excelled in identifying various things and writing about them and putting them in their proper places.',
       'noise fart* talk. "',
       'an indefinite block is appropriate, even for a minor infraction, if you show know signs of discontinuing. — (talk)',
       "i don't understand why we have a screenshot of ap's gui but not ub. can someone remedy this?",
       "hello! some of the people, places or things you have written about in the article nikolas tryfonos may not be sufficiently well-known to merit articles of their own. the wikipedia community welcomes newcomers, and encourages them to become wikipedians. on wikipedia, each user is entitled to a user page in which they can describe themselves, and this article's content may be incorporated into that page. however, to merit inclusion in the encyclopedia proper, a subject must be notable. we encourage you to wr

array(['expert categorizer why be there no mention of the fact that nazi be particularly great categorizer ? they excel in identify various thing and write about they and put they in their proper place .',
       'noise fart * talk . "',
       'an indefinite block be appropriate , even for a minor infraction , if you show know sign of discontinuing . — ( talk )',
       "I do not understand why we have a screenshot of ap 's gui but not ub . can someone remedy this ?",
       "hello ! some of the people , place or thing you have write about in the article nikolas tryfono may not be sufficiently well - know to merit article of their own . the wikipedia community welcome newcomer , and encourage they to become wikipedian . on wikipedia , each user be entitle to a user page in which they can describe themselves , and this article 's content may be incorporate into that page . however , to merit inclusion in the encyclopedia proper , a subject must be notable . we encourage you to write or

CPU times: user 8min 29s, sys: 3.12 s, total: 8min 32s
Wall time: 8min 33s


In [ ]:
# разделим данные на выборки
X_train, X_test, y_train, y_test = train_test_split(
    df['lemmatized_text'],
    df['toxic'],
    test_size=.2,
    random_state=RS
)
display(
    X_train.shape,
    X_test.shape,
    y_train.shape,
    y_test.shape
)

(24000,)

(6000,)

(24000,)

(6000,)

Векторизируем данные:

In [ ]:
# загрузим словарь стоп-слов
nltk.download('stopwords')

# загрузим английские стоп-слова
stopwords = set(nltk_stopwords.words('english'))

# создадим векторизатор, добавим стоп-слова
tf_idf_vectorizer = TfidfVectorizer(
    stop_words=stopwords
)

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
%%time

# векторизируем выборки
features_train_tfidf = tf_idf_vectorizer.fit_transform(X_train)
features_test_tfidf = tf_idf_vectorizer.transform(X_test)

# проверим результат
display(
    features_train_tfidf.shape,
    features_test_tfidf.shape
)

(24000, 53369)

(6000, 53369)

CPU times: user 1.61 s, sys: 16 ms, total: 1.63 s
Wall time: 1.63 s


**Вывод**

Мы успешно обработали данные. Выяснили, что в данных:

- нет пропусков
- корректные типы
- нет дубликатов
- дисбаланс классов: токсичных комментариев около 10%

Далее мы:
- удалили неинформативный столбец Unnamed: 0
- привели слова к нижнему регистру и очистили от лишних символов
- лемматизировали данные
- разделили данные на выборки
- очистили тексты от стоп-слов и векторизировали данные

### Этап 4. Обучение моделей

Обучим модели LogisticRegression и LinearSVC через GridSearchCV. Для борьбы с несбалансированными классами применим гиперпараметр class_weight='balanced':

In [ ]:
# зададим шаги пайплайна
steps = [('vectorizer', tf_idf_vectorizer) , ('classifier', LogisticRegression())]

# зададим модели с гиперпараметрами
params = [
    {'classifier': [LogisticRegression(
        max_iter=1000,
        class_weight='balanced',
        n_jobs=-1
    )],
     'classifier__C': [1, 10, 12, 15]
    },
    {'classifier': [LinearSVC(
         random_state=RS,
         class_weight='balanced',
         max_iter=3000
     )],
      'classifier__C': [1, 10, 12, 15]
     }
]

# напишем функцию для поиска оптимальных гиперпараметров
def pipeline_gridsearchcv(X, y, steps, params):

    # сформируем итоговый pipeline
    pipe = Pipeline(steps)

    # собираем все вместе, используем F1 в качестве метрики
    grid = GridSearchCV(
        pipe,
        param_grid=params,
        cv=5,
        n_jobs=-1,
        scoring='f1',
        verbose=100,
        error_score='raise'
    )
    grid.fit(X, y)

    # посмотрим лучшие гиперпараметры
    print('')
    print('Параметры лучшей модели:', grid.best_params_)
    print('Значение лучшей метрики качества: {:.2f}'.format(grid.best_score_))

    return grid

In [ ]:
%%time

best_model = pipeline_gridsearchcv(X_train, y_train, steps, params)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV 1/5; 1/8] START classifier=LogisticRegression(class_weight='balanced', max_iter=1000, n_jobs=-1), classifier__C=1
[CV 1/5; 1/8] END classifier=LogisticRegression(class_weight='balanced', max_iter=1000, n_jobs=-1), classifier__C=1; total time=  15.3s
[CV 2/5; 1/8] START classifier=LogisticRegression(class_weight='balanced', max_iter=1000, n_jobs=-1), classifier__C=1
[CV 2/5; 1/8] END classifier=LogisticRegression(class_weight='balanced', max_iter=1000, n_jobs=-1), classifier__C=1; total time=  27.0s
[CV 3/5; 1/8] START classifier=LogisticRegression(class_weight='balanced', max_iter=1000, n_jobs=-1), classifier__C=1
[CV 3/5; 1/8] END classifier=LogisticRegression(class_weight='balanced', max_iter=1000, n_jobs=-1), classifier__C=1; total time=  17.5s
[CV 4/5; 1/8] START classifier=LogisticRegression(class_weight='balanced', max_iter=1000, n_jobs=-1), classifier__C=1
[CV 4/5; 1/8] END classifier=LogisticRegression(class_weight

In [ ]:
# проверим модель на тестовой выборке
prediction = best_model.predict(X_test)
f1_score(y_test, prediction)

0.7769423558897243

Модель показала хороший результат на тестовой выборке. Нам удалось превысить целевой порог 0.75 F1-меры.

In [ ]:
# обучим DummyClassifier для проверки адекватности нашей модели
dummy = DummyClassifier(
    random_state=RS,
    strategy='constant',
    constant=1
)
dummy.fit(features_train_tfidf, y_train)
dummy_pred = dummy.predict(features_test_tfidf)

# найдем метрику DummyClassifier
f1_score(y_test, dummy_pred)

0.18401937046004843

Наша модель показала свою адекватность по сравнению с dummy-моделью: метрика F1 значительно превысила показатель DummyClassifier.

**Вывод**

1. Мы обучили две модели: LogisticRegression и LinearSVC. Для борьбы с дисбалансом классов в данных мы применили гиперпараметр class_weight='balanced'
2. Через GridSearchCV мы выявили лучшую модель и гиперпараметры, используя метрику F1: LogisticRegression с регуляризацией C=10
3. Наша модель показала хороший результат на тестовой выборке: F1-мера около 0.78. Это превысило необходимый порог в 0.75
4. Модель показала свою адекватность по сравнению с dummy-моделью: метрика F1 значительно превысила показатель DummyClassifier.

###  Этап 5. Выводы

Мы успешно обработали данные. Выяснили, что в данных:

- нет пропусков
- корректные типы
- нет дубликатов
- дисбаланс классов: токсичных комментариев около 10%

Далее мы:

- удалили неинформативный столбец Unnamed: 0
- привели слова к нижнему регистру и очистили от лишних символов
- лемматизировали данные
- разделили данные на выборки
- очистили тексты от стоп-слов и векторизировали данные

Затем мы обучили модели:

1. Обучили LogisticRegression и LinearSVC. Для борьбы с дисбалансом классов в данных применили гиперпараметр class_weight='balanced'
2. Через GridSearchCV мы выявили лучшую модель и гиперпараметры, используя метрику F1: LogisticRegression с регуляризацией C=10
3. Наша модель показала хороший результат на тестовой выборке: F1-мера около 0.78. Это превысило необходимый порог в 0.75
4. Модель показала свою адекватность по сравнению с dummy-моделью: метрика F1 значительно превысила показатель DummyClassifier.

**В итоге нам удалось достичь поставленной цели и создать качественную модель, которая поможет определять негативные комментарии.**